In [1]:
import os
from collections import defaultdict
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
os.chdir("/master/nplatt/patho_detect")

In [10]:
samples

{'B5_n01_exp4': 1,
 'B1_n04_exp4': 4,
 'B2_n04_exp4': 4,
 'B3_n04_exp4': 4,
 'B4_n04_exp4': 4,
 'B10_n16_exp4': 16,
 'B11_n16_exp4': 16,
 'B12_n16_exp4': 16,
 'B13_n16_exp4': 16,
 'B14_n16_exp4': 16,
 'B15_n16_exp4': 16,
 'B17_n16_exp4': 16,
 'B18_n16_exp4': 16,
 'B19_n16_exp4': 16,
 'B20_n16_exp4': 16,
 'B21_n16_exp4': 16,
 'B6_n16_exp4': 16,
 'B7_n16_exp4': 16,
 'B8_n16_exp4': 16,
 'B9_n16_exp4': 16,
 'Control_exp4': 'C'}

In [13]:
#list samples including the number of samples per pool

samples = { 'B5_n01_exp4':  1,
            'B1_n04_exp4':  4,
            'B2_n04_exp4':  4,
            'B3_n04_exp4':  4,
            'B4_n04_exp4':  4,
            'B10_n16_exp4': 16,
            'B11_n16_exp4': 16,
            'B12_n16_exp4': 16,
            'B13_n16_exp4': 16,
            'B14_n16_exp4': 16,
            'B15_n16_exp4': 16,
            'B17_n16_exp4': 16,
            'B18_n16_exp4': 16,
            'B19_n16_exp4': 16,
            'B20_n16_exp4': 16,
            'B21_n16_exp4': 16,
            'B6_n16_exp4':  16,
            'B7_n16_exp4':  16,
            'B8_n16_exp4':  16,
            'B9_n16_exp4':  16,
            'Control_exp4': "C"}

genomes = [ "mycTub_GCF_000195955",
            "mycBov_GCF_005156105",
            "schBov_GCA_003958945",
            "schMan_GCF_000237925",
            "plaFal_GCF_000002765",
            "plaViv_GCF_000002415",
            "musMus_GCF_000001635",
            "homSap_GCF_000001405" ]

In [11]:
os.mkdir("results/figs_tables")

In [36]:
df

,sample,n_pooled,mycTub_GCF_000195955,mycBov_GCF_005156105,schBov_GCA_003958945,schMan_GCF_000237925,plaFal_GCF_000002765,plaViv_GCF_000002415,musMus_GCF_000001635,homSap_GCF_000001405
0,B5_n01_exp4,1,94216,13513178,1439056,1282500,25470998,13098022,98039892,17736
1,B1_n04_exp4,4,28126,3911286,632244,551784,6298594,3204834,21292418,4748
2,B2_n04_exp4,4,25736,3704584,588016,497360,5520544,2952706,20409604,4526
3,B3_n04_exp4,4,38452,5174174,923394,810974,9227534,4629094,31004454,6112
4,B4_n04_exp4,4,31238,4665788,991742,865362,9130428,4233654,29022056,5532
5,B10_n16_exp4,16,9818,1381884,309830,255248,2584178,1252476,9271802,2546
6,B11_n16_exp4,16,9306,1125792,197986,168172,1699986,863044,6206212,2190
7,B12_n16_exp4,16,6762,997722,188610,162284,1399256,756114,5541346,1754
8,B13_n16_exp4,16,6638,1183720,227944,188208,1409648,849098,6712370,1966
9,B14_n16_exp4,16,9084,1173048,245554,207954,1509244,831452,6520846,2124


In [39]:
#read in bbmap data


df=pd.DataFrame(columns=["sample",   "n_pooled"] + genomes)
                
                
for sample in samples.keys():
    n_pooled=samples[sample]
    
    with open ("results/bbmap/20201104_pool/{}_refstats.file".format(sample), 'r') as infile:
        p_unambiguous_reads={}
        unambiguous_mb={}
        p_ambiguous_reads={}
        ambiguous_mb={}
        n_unambiguous_reads={} 
        n_ambiguous_reads={}
        assigned_reads={}   
        assigned_bases={}

        next(infile)
        for entry in infile:
            genome, *data = entry.rstrip().split("\t")
            
            ( p_unambiguous_reads[genome],
              unambiguous_mb[genome],
              p_ambiguous_reads[genome],
              ambiguous_mb[genome],
              n_unambiguous_reads[genome],
              n_ambiguous_reads[genome],
              assigned_reads[genome],
              assigned_bases[genome] ) = data
            
        data=[sample, n_pooled]
        for genome in genomes:
            data.append(assigned_reads[genome])

        df = df.append(pd.Series(data, index = df.columns), ignore_index=True)
        
df.to_csv("results/figs_tables/bbmap_assigned_reads.csv", index = False)

In [ ]:
markers={"C": "P",
    1: "O",
    4: "s",
    16: "*"}

fig = plt.gcf() 
fig.set_size_inches(8, 8) 
    
for sample in df.itterrows():
    
    plt.scatter(record.POS, record.INFO["AF"], 
                s=30, 
                color="black", #color=colors[mut_type], 
                edgecolor='black', 
                linewidths=0.5)

    #plt.ylim(0, 1)
    #plt.xlim(0,30e3)
    #plt.ylabel("{} alt. AF".format(sample.upper()))
    #plt.xlabel("SARS-CoV-2 position ({})".format(record.CHROM))
    #plt.savefig("results/figs_tables/lolipops/{}_lolipop.svg".format(sample)) 
    
    plt.show()
    plt.close()

    
   